# Задача: система має відповідати на питання користувача по контенту документу.


# Документи і приклади запитань:
1. https://assets.ctfassets.net/fzn2n1nzq965/5Qj1NdQFtWO3rIE4WhAVl9/21202f8455f5a877444f91c1b48baf6e/stripe-2022-update.pdf
- Where are the top startups located?
2. https://www.coffeeb.com/media/77/c5/ab/1660233188/CoffeeB_Manual%20Gl
obe_EN_10.08.2022.pdf
- How can I resolve the issue of my coffee being trapped in the machine?

# Project stack
llama-index => 0.9
pypdf
chromaDB

In [2]:
!pip install llama-index pypdf chromadb

     -------------------------------------- 286.1/286.1 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 525.5/525.5 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 15.3/15.3 MB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 2.2 MB/s eta 0:00:00
     -------------------------------------- 395.2/395.2 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 151.6/151.6 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 92.1/92.1 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 60.8/60.8 kB ? eta 0:00:00
     ---------------------------------------- 15.8/15.8 MB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 41.3/41.3 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 3.4/3.4 MB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 58.4/58


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
%pip install llama-index-vector-stores-chroma

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Set up openai token

In [27]:
import os

OPENAI_API_KEY = "<YOUR_API_KEY>"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [28]:
# Defining ChatGPT model
OPENAI_MODEL = "gpt-3.5-turbo-16k"

# ChatGPT completion setup
OPENAI_COMPLETION_OPTIONS = {
    "temperature": 0.1,  # respond to accuracy of llm (from 0.1 up to 2)
    "max_tokens": 1000,  # max amount of tokens that llm is uses 
    "top_p": 1,  # top value of temperature  
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "request_timeout": 60.0,
}

# Base prompt
LLM_BASE_PROMPT = ""

# Create service context

In [29]:
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager
from llama_index.core import PromptHelper, ServiceContext
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

# Create instance of llm
llm = OpenAI(model=OPENAI_MODEL, temperature=0)
# Define embeddings model 
embed_model = OpenAIEmbedding()

# Set up Node parser
node_parser = SimpleNodeParser.from_defaults(
    chunk_size=1024,
    chunk_overlap=20
)

prompt_helper = PromptHelper(
    context_window=4096,
    num_output=256,
    chunk_overlap_ratio=0.1,
    chunk_size_limit=None
)

# Logging
llama_debug = LlamaDebugHandler()
callback_manager = CallbackManager([llama_debug])

service_context = {
    "llm": llm,
    "embed_model": embed_model,
    "node_parser": node_parser,
    "prompt_helper": prompt_helper,
    "system_prompt": LLM_BASE_PROMPT,
    "callback_manager": callback_manager
}

# Create client and a new collection

In [30]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
import chromadb

db2 = chromadb.PersistentClient(path="./chroma_db")

chroma_stripe_collection = db2.get_or_create_collection("stripe_embeddings")
chroma_coffeB_collection = db2.get_or_create_collection("coffeB_embeddings")

# load documents
stripe_documents = SimpleDirectoryReader("./data/stripe/").load_data()
coffeB_documents = SimpleDirectoryReader("./data/coffeB/").load_data()

# set up ChromaVectorStore and load in data
stripe_vector_store = ChromaVectorStore(chroma_collection=chroma_stripe_collection)
stripe_storage_context = StorageContext.from_defaults(vector_store=stripe_vector_store)
stripe_index = VectorStoreIndex.from_documents(
    stripe_documents, **service_context, storage_context=stripe_storage_context
)

coffeB_vector_store = ChromaVectorStore(chroma_collection=chroma_coffeB_collection)
coffeB_storage_context = StorageContext.from_defaults(vector_store=coffeB_vector_store)
coffeB_index = VectorStoreIndex.from_documents(
    coffeB_documents, **service_context, storage_context=coffeB_storage_context
)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING ->  1.417562 seconds
**********
**********
Trace: index_construction
    |_CBEventType.EMBEDDING ->  0.940385 seconds
**********


# Creating query engine and tools

In [33]:
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.tools import QueryEngineTool

# initialize tools
list_tool = QueryEngineTool.from_defaults(
    query_engine=stripe_index.as_query_engine(),
    description="Useful for retrieving information about Stripe.",
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=coffeB_index.as_query_engine(),
    description="Manual for coffeB coffe machine. Retrieves full guidance how to use coffeB machine.",
)

# initialize router query engine (single selection, pydantic)
query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

# Logining

In [36]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [37]:
from IPython.display import Markdown, display
response = query_engine.query("Where are the top startups located?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: Stripe is a popular payment processing platform used by many startups, so information about Stripe may provide insights into where top startups are located..
Selecting query engine 0: Stripe is a popular payment processing platform used by many startups, so information about Stripe may provide insights into where top startups are located..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>The top startups are located in the Bay Area, Los Angeles, Austin, New York City, Miami, London, Singapore, Paris, Tokyo, and Toronto.</b>

In [41]:
response = query_engine.query("What is stripe?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: Stripe is a service that provides information about Stripe, which is relevant to the question 'What is Stripe?'.
Selecting query engine 0: Stripe is a service that provides information about Stripe, which is relevant to the question 'What is Stripe?'.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Stripe is a company that focuses on improving online payment processes by addressing deficiencies in the current payment systems. They invest in security measures at various levels of their operations, including data encryption, access controls, and real-time transaction monitoring. Additionally, Stripe prioritizes reliability in their financial operations by holding funds with systematically important financial institutions and continuously monitoring their financial partners and issuers.</b>

In [42]:
response = query_engine.query("How stripe is grew up?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: The question is about the growth of Stripe, and choice (1) is specifically mentioned as useful for retrieving information about Stripe..
Selecting query engine 0: The question is about the growth of Stripe, and choice (1) is specifically mentioned as useful for retrieving information about Stripe..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Stripe experienced significant growth with the number of new businesses joining the platform increasing by 19% in 2022. On average, more than 1,000 new ventures were launched every day. Additionally, Stripe now supports businesses in over 50 countries, with 55% of the businesses that joined last year being based outside of the U.S.</b>

In [43]:
response = query_engine.query("Tell me about stripe payments")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: The choice 'Useful for retrieving information about Stripe' is the most relevant to the question 'Tell me about stripe payments'..
Selecting query engine 0: The choice 'Useful for retrieving information about Stripe' is the most relevant to the question 'Tell me about stripe payments'..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Stripe payments have been a focus for improvement, with Stripe working to address deficiencies in online payment processes. They have introduced highly optimized checkout products, such as Payment Element, which have shown to increase revenue for businesses that adopt them. These improvements have significantly reduced checkout times for buyers through various UI adjustments and cross-device optimizations. Additionally, Stripe has updated its "Remember Me" functionality to enhance user experience during payments.</b>

In [44]:
response = query_engine.query("How much people are active stripe users?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: The question is related to retrieving information about Stripe, which is mentioned in choice (1)..
Selecting query engine 0: The question is related to retrieving information about Stripe, which is mentioned in choice (1)..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>There are now more than 100,000 active in-person devices worldwide.</b>

In [45]:
response = query_engine.query("How coffe machine works?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 1: The manual for the coffeB coffee machine provides full guidance on how to use the machine, which includes information on how the coffee machine works..
Selecting query engine 1: The manual for the coffeB coffee machine provides full guidance on how to use the machine, which includes information on how the coffee machine works..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>The coffee machine should be placed on a dry, water-proof, and heat-proof surface. It should not be placed directly on marble or wood. The machine needs to be at a specific distance from the sink and open flame. The water tank should be cleaned and filled with cold water, and the power plug should be inserted into an earthed socket. After ensuring the control lever is pressed down, the machine needs to be allowed to warm up for 30 minutes at room temperature. For coffee preparation, one coffee ball is needed for each cup of coffee. The drip tray should be adjusted based on the cup size, and the coffee ball should be placed in the slot before pressing the control lever down to dispense the coffee.</b>

In [46]:
response = query_engine.query("Requirements for using coffeB ?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 1: The manual for the coffeB coffee machine provides full guidance on how to use the machine, which would include the requirements for using it effectively..
Selecting query engine 1: The manual for the coffeB coffee machine provides full guidance on how to use the machine, which would include the requirements for using it effectively..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Requirements for using coffeB include placing the machine on a dry, horizontal, stable, and flat surface that is resistant to heat and liquids. The machine should not be placed inside a cupboard or on a hot surface. It is important to maintain specific distances from the sink and open flames, and the machine should not be near hobs, gas cookers, or naked flames. Additionally, the power cable should not be accessible to children under 8 years old, and the machine should not be placed on marble surfaces or untreated/oiled woodwork.</b>

In [47]:
response = query_engine.query("How can I resolve the issue of my coffee being trapped in the machine?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 1: The manual for the coffee machine provides full guidance on how to use the coffee machine, including troubleshooting steps for resolving issues like coffee being trapped in the machine..
Selecting query engine 1: The manual for the coffee machine provides full guidance on how to use the coffee machine, including troubleshooting steps for resolving issues like coffee being trapped in the machine..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Press the control lever down and start the coffee brewing process. As soon as coffee is dispensed, cancel the dispensing process by pressing the coffee key again. If the coffee ball has not been discarded, carefully scrape the coffee ball out of the machine with your finger and/or a spoon. Flush out the pipings if needed. If two balls are stuck, push them down with your finger and remove them.</b>